<a href="https://colab.research.google.com/github/majimpala/A1111-Web-UI-Installer/blob/main/tts_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Fast install, might break in the future.
!pip install 'sphn<0.2'
!pip install --no-deps "moshi==0.2.7"
# Slow install (will download torch and cuda), but future proof.
# !pip install "moshi==0.2.7"

In [2]:
import argparse
import sys

import numpy as np
import torch
from moshi.models.loaders import CheckpointInfo
from moshi.models.tts import DEFAULT_DSM_TTS_REPO, DEFAULT_DSM_TTS_VOICE_REPO, TTSModel

from IPython.display import display, Audio

In [3]:
text = """
Hey there! How are you? I had the craziest day today.
"""
voice = "expresso/ex01-ex02_whisper_001_channel2_717s.wav"
print(f"See https://huggingface.co/{DEFAULT_DSM_TTS_VOICE_REPO} for available voices.")

See https://huggingface.co/kyutai/tts-voices for available voices.


In [4]:
# Set everything up
checkpoint_info = CheckpointInfo.from_hf_repo(DEFAULT_DSM_TTS_REPO)
tts_model = TTSModel.from_checkpoint_info(
    checkpoint_info, n_q=32, temp=0.6, device=torch.device("cuda")
)

# If you want to make a dialog, you can pass more than one turn [text_speaker_1, text_speaker_2, text_2_speaker_1, ...]
entries = tts_model.prepare_script([text], padding_between=1)
voice_path = tts_model.get_voice_path(voice)
# CFG coef goes here because the model was trained with CFG distillation,
# so it's not _actually_ doing CFG at inference time.
# Also, if you are generating a dialog, you should have two voices in the list.
condition_attributes = tts_model.make_condition_attributes(
    [voice_path], cfg_coef=2.0
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
print("Generating audio...")

pcms = []
def _on_frame(frame):
    print("Step", len(pcms), end="\r")
    if (frame != -1).all():
        pcm = tts_model.mimi.decode(frame[:, 1:, :]).cpu().numpy()
        pcms.append(np.clip(pcm[0, 0], -1, 1))

# You could also generate multiple audios at once by extending the following lists.
all_entries = [entries]
all_condition_attributes = [condition_attributes]
with tts_model.mimi.streaming(len(all_entries)):
    result = tts_model.generate(all_entries, all_condition_attributes, on_frame=_on_frame)

print("Done generating.")
audio = np.concatenate(pcms, axis=-1)

Generating audio...


/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
W0706 21:00:43.876000 46322 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(


Done generating.


In [6]:
display(
    Audio(audio, rate=tts_model.mimi.sample_rate, autoplay=True)
)

# Task
Modify the code to have a user interface for text-to-speech generation.

## Import necessary libraries

### Subtask:
Import `ipywidgets` and `IPython.display` for creating and displaying the UI elements.


**Reasoning**:
The subtask requires importing `ipywidgets` and `IPython.display`. These imports are needed to create and display the user interface elements in subsequent steps.



In [7]:
import ipywidgets as widgets
from IPython.display import display

## Create ui elements

### Subtask:
Generate code to create widgets for text input, voice selection (e.g., a dropdown or radio buttons), and a button to trigger the audio generation.


**Reasoning**:
Create the specified ipywidgets for text input, voice selection, and audio generation trigger button based on the instructions.



In [8]:
text_input = widgets.Textarea(
    value="Hey there! How are you? I had the craziest day today.",
    description='Text:',
    disabled=False,
    layout={'width': 'auto', 'height': '200px'}
)

# Using the 'voice' variable from previous execution. In a real application,
# you would dynamically fetch available voices from the model or a list.
available_voices = [voice, "dummy/voice1.wav", "dummy/voice2.wav"] # Add dummy options for demonstration
voice_dropdown = widgets.Dropdown(
    options=available_voices,
    value=voice,
    description='Voice:',
    disabled=False,
)

generate_button = widgets.Button(
    description='Generate Audio',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to generate audio',
    icon='play'
)

display(text_input)
display(voice_dropdown)
display(generate_button)

Textarea(value='Hey there! How are you? I had the craziest day today.', description='Text:', layout=Layout(hei…

Dropdown(description='Voice:', options=('expresso/ex01-ex02_whisper_001_channel2_717s.wav', 'dummy/voice1.wav'…

Button(description='Generate Audio', icon='play', style=ButtonStyle(), tooltip='Click to generate audio')

## Modify the generation function

### Subtask:
Adapt the existing audio generation code into a function that takes the text and voice selection from the UI as input.


**Reasoning**:
The task is to adapt the existing audio generation code into a function. I will define the function and move the relevant code inside, replacing the hardcoded values with the function arguments.



In [9]:
def generate_audio_from_ui(input_text, selected_voice):
    """
    Generates audio from text using the TTS model with a selected voice.

    Args:
        input_text: The text to synthesize.
        selected_voice: The path to the voice file.

    Returns:
        A NumPy array containing the generated audio.
    """
    # Set everything up
    # Assuming checkpoint_info and tts_model are already initialized globally
    # checkpoint_info = CheckpointInfo.from_hf_repo(DEFAULT_DSM_TTS_REPO)
    # tts_model = TTSModel.from_checkpoint_info(
    #     checkpoint_info, n_q=32, temp=0.6, device=torch.device("cuda")
    # )

    # If you want to make a dialog, you can pass more than one turn [text_speaker_1, text_speaker_2, text_2_speaker_1, ...]
    entries = tts_model.prepare_script([input_text], padding_between=1)
    voice_path = tts_model.get_voice_path(selected_voice)
    # CFG coef goes here because the model was trained with CFG distillation,
    # so it's not _actually_ doing CFG at inference time.
    # Also, if you are generating a dialog, you should have two voices in the list.
    condition_attributes = tts_model.make_condition_attributes(
        [voice_path], cfg_coef=2.0
    )

    print("Generating audio...")

    pcms = []
    def _on_frame(frame):
        # print("Step", len(pcms), end="\r") # Removed for cleaner output during UI interaction
        if (frame != -1).all():
            pcm = tts_model.mimi.decode(frame[:, 1:, :]).cpu().numpy()
            pcms.append(np.clip(pcm[0, 0], -1, 1))

    # You could also generate multiple audios at once by extending the following lists.
    all_entries = [entries]
    all_condition_attributes = [condition_attributes]
    with tts_model.mimi.streaming(len(all_entries)):
        result = tts_model.generate(all_entries, all_condition_attributes, on_frame=_on_frame)

    # print("Done generating.") # Removed for cleaner output during UI interaction
    audio = np.concatenate(pcms, axis=-1)
    return audio

# Note: tts_model is expected to be initialized in a previous cell and available globally.

## Link ui elements to the function

### Subtask:
Connect the button's click event to the generation function so that the audio is generated when the button is clicked.


**Reasoning**:
Define the function to handle the button click, call the generation function within it, and link the button's click event to this handler function.



In [10]:
output_widget = widgets.Output()

def on_generate_button_clicked(b):
    """Handles the click event of the generate button."""
    with output_widget:
        output_widget.clear_output()
        print("Generating audio...")
        generated_audio = generate_audio_from_ui(text_input.value, voice_dropdown.value)
        print("Done generating.")
        display(Audio(generated_audio, rate=tts_model.mimi.sample_rate, autoplay=True))

generate_button.on_click(on_generate_button_clicked)

display(output_widget)

Output()

## Summary:

### Data Analysis Key Findings

*   The code successfully created and displayed `ipywidgets` for text input (`Textarea`), voice selection (`Dropdown`), and triggering audio generation (`Button`).
*   The existing audio generation logic was successfully wrapped into a Python function `generate_audio_from_ui` that accepts input text and selected voice as arguments.
*   The click event of the "Generate Audio" button was successfully linked to the `on_generate_button_clicked` function, which retrieves values from the UI widgets, calls the audio generation function, and displays the resulting audio using the `IPython.display.Audio` widget within an `ipywidgets.Output` widget.

### Insights or Next Steps

*   Consider adding error handling and visual feedback (e.g., disabling the button, displaying a progress indicator) during the audio generation process to improve the user experience.
*   Explore dynamically populating the voice dropdown with available voices from the TTS model or a predefined list instead of using hardcoded and dummy options.
